In [128]:
# pip install tqdm
!pip install scikit-image
!pip  install openpyxl
import os
import random
import shutil
from PIL import Image
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from PIL import Image
# !pip install -U scikit-image
from joblib import dump, load
from skimage.feature import graycomatrix, graycoprops
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
from transformers import CLIPProcessor, CLIPModel
import torch
model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor_clip = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [129]:
import os
# BASE_DIR = os.path.dirname(os.path.join(os.path.dirname(__file__), 'pcos'))
# print("Base Dir:", BASE_DIR)
training_dataset_path = '/content/drive/MyDrive/data/input/PCOSGen-train'
test_dataset_path = '/content/drive/MyDrive/data/input/PCOSGen-test/images/'
label_path ='/content/drive/MyDrive/data/input/class_label.xlsx'
data_dir = '/content/drive/MyDrive/data/output'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
train_labels_path = os.path.join(data_dir, 'train_labels.csv' )
val_labels_path = os.path.join(data_dir, 'val_labels.csv' )
models_path = os.path.join(data_dir, 'models')
plot_dir = os.path.join(data_dir, 'plots')
os.makedirs(models_path, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)
result_submission_file =  os.path.join(data_dir, 'result_submission.xlsx' )


In [130]:
import random
import shutil

import pandas as pd
from sklearn.model_selection import train_test_split

# !pip install -U scikit-image
# from configs import *


def prepare_balanced_dataset():
    def load_images_from_folder(folder, category, image_category_map):
        images = []
        for filename in os.listdir(folder):
            if image_category_map.get(filename) == category:
                img_path = os.path.join(folder, filename)
                if os.path.isfile(img_path):
                    images.append(filename)
        return images

    def oversample_images(images, desired_count):
        oversampled = images[:]
        while len(oversampled) < desired_count:
            oversampled.append(random.choice(images))
        return oversampled

    def create_or_clear_dir(dir_path):
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
        os.makedirs(dir_path, exist_ok=True)


    print("Start preparing data...")
    directories = [train_dir, val_dir]
    for directory in directories:
        create_or_clear_dir(directory)

    # Read the Excel file
    df = pd.read_excel(label_path)
    image_category_map = pd.Series(df.Healthy.values, index=df.imagePath).to_dict()

    # Get unique categories and max count
    categories = df['Healthy'].unique()
    max_count = df['Healthy'].value_counts().max()

    # Prepare data
    all_data = []
    for category in categories:
        images = load_images_from_folder(training_dataset_path, category, image_category_map)
        oversampled_images = oversample_images(images, max_count)
        all_data.extend([(image, category) for image in oversampled_images])

    # Split data into training and val sets (80:20)
    train_data, val_data = train_test_split(all_data, test_size=0.2, stratify=[label for _, label in all_data])

    # Function to save images and return label data
    def save_images(data, root_dir):
        label_data = []
        for img_filename, label in data:
            source_path = os.path.join(training_dataset_path, img_filename)
            target_path = os.path.join(root_dir, img_filename)
            shutil.copy(source_path, target_path)
            label_data.append([img_filename, label])
        return label_data

    # Save images and get label data for CSV
    train_label_data = save_images(train_data, train_dir)
    val_label_data = save_images(val_data, val_dir)

    # Save label data to CSV
    pd.DataFrame(train_label_data, columns=['id', 'label']).to_csv(train_labels_path, index=False)
    pd.DataFrame(val_label_data, columns=['id', 'label']).to_csv(val_labels_path, index=False)
    print(f"Balanced Dataset created under {directories}")


In [131]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from torchvision.transforms.functional import to_pil_image

from PIL import Image



def plot_cam(model, image_tensor, target_layer, filename):
    # Switch the model to evaluation mode
    model.eval()
    # Ensure the tensor is on the same device as the model
    image_tensor = image_tensor.to(device)

    with torch.no_grad():
        # Extract features and model output
        features_fn = torch.nn.Sequential(*list(model.base_model.children())[:-2])
        features = features_fn(image_tensor.unsqueeze(0))
        output = model(image_tensor.unsqueeze(0))

    # Move tensors to CPU for further processing
    features = features.cpu()
    output = output.cpu()

    # Get weights for the target class from the fully connected layer
    params = list(model.fc.parameters())
    weight_softmax = np.squeeze(params[0].cpu().data.numpy())

    # Compute CAM
    class_idx = torch.topk(output, 1)[1].int()
    cam = weight_softmax[class_idx.squeeze().item()]
    cam = np.dot(cam, features.reshape((features.shape[1], -1)))
    cam = cam.reshape(features.shape[2], features.shape[3])
    cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))
    cam_img = np.uint8(255 * cam)

    # Resize CAM to the size of the input image
    cam_img = np.float32(Image.fromarray(cam_img).resize((image_tensor.shape[2], image_tensor.shape[1]), Image.LINEAR))

    # Convert CAM to a heatmap
    heatmap = plt.get_cmap('jet')(cam_img)[:, :, :3]
    heatmap = Image.fromarray(np.uint8(heatmap * 255))

    # Create a matplotlib figure
    fig, ax = plt.subplots()
    ax.imshow(transforms.ToPILImage()(image_tensor.cpu().squeeze()))
    ax.imshow(np.asarray(heatmap), cmap='jet', alpha=0.5, extent=(0, image_tensor.shape[2], image_tensor.shape[1], 0))
    ax.axis('off')

    # Save the figure with 600 DPI
    fig.savefig(filename, dpi=600, bbox_inches='tight', pad_inches=0)
    plt.close(fig)



def plot_occlusion(model, image_tensor, filename, occlusion_size=4, stride=1):


    # Ensure the image tensor is on the correct device
    image_tensor = image_tensor.to(device)

    # Calculate the output height and width based on occlusion size and stride
    output_height = int(np.ceil((224 - occlusion_size) / stride) + 1)
    output_width = int(np.ceil((224 - occlusion_size) / stride) + 1)

    # Initialize the heatmap tensor on the correct device
    heatmap = torch.zeros((output_height, output_width), device=device)

    model.eval()
    with torch.no_grad():
        for h in range(0, 224, stride):
            for w in range(0, 224, stride):
                h_start = h
                w_start = w
                h_end = min(224, h + occlusion_size)
                w_end = min(224, w + occlusion_size)

                if (h_end - h_start) != occlusion_size or (w_end - w_start) != occlusion_size:
                    continue

                input_image = image_tensor.clone().detach()
                input_image[:, h_start:h_end, w_start:w_end] = 0
                input_image = input_image.to(device)  # Ensure the modified tensor is on the correct device

                output = model(input_image.unsqueeze(0))
                output = torch.nn.functional.softmax(output, dim=1)
                prob = output.max().item()

                heatmap[int(np.floor(h / stride)), int(np.floor(w / stride))] = prob

    # Plot and save the heatmap
    fig, ax = plt.subplots()
    ax.imshow(image_tensor.cpu().squeeze().permute(1, 2, 0))
    ax.imshow(heatmap.cpu(), cmap='hot', alpha=0.5, extent=(0, 224, 224, 0))
    ax.axis('off')
    plt.savefig(filename, dpi=600)
    plt.close()


!pip install shap

import shap
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

import shap
import torch
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

# Ensure the SHAP DeepExplainer and model are compatible

import shap
import torch
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

# Set up the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to plot SHAP values
def plot_shap(model, image_tensor, filename):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Ensure the model is in evaluation mode
    model.eval()

    # Check if the image tensor has a batch dimension, and add it if missing
    if image_tensor.dim() == 3:
        # Adds a batch dimension at the start
        image_tensor = image_tensor.unsqueeze(0)

    # Move the image tensor to the same device as the model
    image_tensor = image_tensor.to(device)

    # Define a wrapper function for the model to be compatible with SHAP
    def model_wrapper(input_data):
        # Convert input_data to a PyTorch tensor if not already, ensuring it is on the correct device
        input_data = torch.tensor(input_data, device=device, dtype=torch.float32)
        # Ensure the input data has the correct shape (batch_size, channels, height, width)
        if input_data.dim() == 3:
            input_data = input_data.unsqueeze(0)
        # Forward pass through the model
        with torch.no_grad():
            output = model(input_data)
        # Convert output to numpy array for SHAP
        return output.cpu().numpy()

    # Assuming the image tensor shape is [batch_size, channels, height, width], like [1, 3, 224, 224]
    input_shape = image_tensor.shape[1:]  # This gets the shape as (channels, height, width)

    # Create a SHAP masker specifying the expected input shape
    masker = shap.maskers.Image("input", shape=input_shape)

    # Create a SHAP explainer
    explainer = shap.Explainer(model_wrapper, masker)

    # Generate SHAP values
    shap_values = explainer(image_tensor.cpu().numpy())

    # Plotting the SHAP values
    # Depending on your specific case, you might need to adjust how you index into shap_values
    shap.image_plot(shap_values[0], -image_tensor.cpu().numpy())

    # Save the plot
    plt.savefig(filename)
    plt.close()

In [133]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models

# from configs import data_dir, train_dir, train_labels_path, plot_dir
# from configs import models_path
# from feature_interpretability import plot_cam, plot_occlusion

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 32


class PCOSDataset(Dataset):
    def __init__(self, data_dir, labels_file_path=None, transform=None):
        self.transform = transform
        if labels_file_path==None:
            files = os.listdir(data_dir)
            self.df = pd.DataFrame({'id':files, 'label':[0]*len(files)})
        else:
            self.df = pd.read_csv(labels_file_path)
        self.images_dir = data_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id, label = self.df.iloc[idx]
        img_path = os.path.join(self.images_dir, f"{img_id}")  # Adjust file extension as needed
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        return image, label


transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5)
    ], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


class ModifiedResNet(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet, self).__init__()
        self.base_model = models.resnet50(weights=True)
        # Modify the existing ResNet model to include SEBlocks
        self.se_block1 = SEBlock(256)
        self.se_block2 = SEBlock(512)
        self.se_block3 = SEBlock(1024)
        self.se_block4 = SEBlock(2048)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.base_model.conv1(x)
        x = self.base_model.bn1(x)
        x = self.base_model.relu(x)
        x = self.base_model.maxpool(x)

        x = self.base_model.layer1(x)
        x = self.se_block1(x)
        x = self.base_model.layer2(x)
        x = self.se_block2(x)
        x = self.base_model.layer3(x)
        x = self.se_block3(x)
        x = self.base_model.layer4(x)
        x = self.se_block4(x)

        x = self.base_model.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x


def dnn_model_train_and_eval():
    # Create dataset instances
    train_dataset = PCOSDataset(data_dir=train_dir, labels_file_path=train_labels_path, transform=transformations)
    # Split train dataset
    len_train = int(0.8 * len(train_dataset))
    len_val = len(train_dataset) - len_train
    train_ds, val_ds = random_split(train_dataset, [len_train, len_val])

    # DataLoaders

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    # training and evaluation ==============================================================================================
    def train_epoch(model, dataloader, loss_fn, optimizer, device):
        model.train()
        total_loss, total_correct = 0, 0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == targets).sum().item()
        return total_loss / len(dataloader.dataset), total_correct / len(dataloader.dataset)

    def evaluate(model, dataloader, loss_fn, device):
        model.eval()
        total_loss, total_correct = 0, 0
        with torch.no_grad():
            for inputs, targets in dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                total_loss += loss.item()
                total_correct += (outputs.argmax(1) == targets).sum().item()
        return total_loss / len(dataloader.dataset), total_correct / len(dataloader.dataset)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ModifiedResNet(num_classes=2).to(device)
    optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    loss_fn = nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []
    early_stopping_patience = 20
    best_val_loss = float('inf')
    patience = 0

    epochs = 150
    for epoch in range(epochs):
        print(f"Start epoch {epoch}")
        train_loss, train_acc = train_epoch(model, train_dl, loss_fn, optimizer, device)
        val_loss, val_acc = evaluate(model, val_dl, loss_fn, device)
        print(
            f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

        # Store metrics for plotting
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        scheduler.step()  # Adjust the learning rate based on scheduler

        # Early Stopping Check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience = 0
        else:
            patience += 1
            if patience >= early_stopping_patience:
                print("Early stopping triggered")
                break

    # Plotting
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.legend()
    plt.title('Loss over Epochs')
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy over Epochs')
    plt.show()
    # Save the model
    torch.save(model.state_dict(), os.path.join(models_path, 'dnn_model.pth'))
    # plot_interpretability_plots(model, val_dl, plot_dir, 'val')
    # Assuming val_dl is your DataLoader for the validation dataset
    # top_images_info = get_top_images_and_predictions(model, val_dl, device, top_k=5)
    # save_top_images_with_predictions(top_images_info, plot_dir)
    return model


def dnn_predict(image_dir, labels_file_path=None, loaded_model=None, dtype='val'):
    # Ensure the model architecture is defined exactly as before
    if loaded_model == None:
        loaded_model = ModifiedResNet(num_classes=2).to(device)
        loaded_model.load_state_dict(torch.load(os.path.join(models_path, 'dnn_model.pth')))
        # Make sure to call eval() for inference to set the model to evaluation mode
        # loaded_model.eval()
    test_dataset = PCOSDataset(data_dir=image_dir, labels_file_path=labels_file_path, transform=transformations)
    test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Load the model parameters from the saved file


    def predict_(model, dataloader, device):
        model.eval()  # Set the model to evaluation mode
        predictions = []
        with torch.no_grad():  # Disable gradient computation
            for inputs, _ in dataloader:
                inputs = inputs.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                predictions.extend(predicted.cpu().numpy())
        return predictions

    # Assuming test_dl is your DataLoader for the test dataset
    predictions = predict_(loaded_model, test_dl, device)
    plot_interpretability_plots(loaded_model, test_dl, plot_dir, dtype)
    # top_images_info = get_top_images_and_predictions(loaded_model, test_dl, device, top_k=5)
    # save_top_images_with_predictions(top_images_info, plot_dir, 'test')
    return predictions


def plot_interpretability_plots(model, dataloader, output_dir, type='val', no_of_plot=20):
    model.eval()
    top_images = []  # Store top images
    with torch.no_grad():
        for images, _ in dataloader:
            outputs = model(images.to(device))
            _, preds = torch.max(outputs, 1)
            top_images.extend(images.cpu())
            if len(top_images) >= no_of_plot:  # Only keep top 5 images
                break

    for i, image_tensor in enumerate(top_images[:no_of_plot]):
        cam_filename = f"{output_dir}/{type}_cam_plot_{i + 1}.png"
        occlusion_filename = f"{output_dir}/{type}_occlusion_plot_{i + 1}.png"
        shap_filename  = f"{output_dir}/{type}_shap_plot_{i + 1}.png"
        # Generate and save CAM plot
        plot_cam(model, image_tensor, target_layer=model.base_model.layer4[-1], filename=cam_filename)

        # Generate and save Occlusion plot
        # plot_occlusion(model, image_tensor, filename=occlusion_filename)
        # plot_shap(model, image_tensor, shap_filename)







def get_top_images_and_predictions(model, dataloader, device, top_k=5):
    model.eval()
    top_images_info = []

    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)

            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            top_prob, top_pred_classes = torch.topk(probabilities, k=1, dim=1)

            for i in range(images.size(0)):
                top_images_info.append((images[i].cpu(), top_pred_classes[i].cpu(), top_prob[i].cpu()))

    # Sort by confidence and select top_k
    top_images_info.sort(key=lambda x: x[2], reverse=True)
    return top_images_info[:top_k]


from PIL import ImageDraw, ImageFont


def save_top_images_with_predictions(top_images_info, output_dir, type='val'):
    os.makedirs(output_dir, exist_ok=True)

    # Optional: specify a font for the text on the image
    # font = ImageFont.truetype(<font-file>, 24)

    for i, (image_tensor, pred_class, _) in enumerate(top_images_info):
        image = transforms.ToPILImage()(image_tensor)
        draw = ImageDraw.Draw(image)

        # Write prediction on the image
        # If you have a font specified, add `, font=font` to the text() call
        text = f'Pred: {pred_class.item()}'
        draw.text((10, 10), text, fill='red')

        # Save the image
        image.save(os.path.join(output_dir, f"{type}_top_image_{i + 1}.png"))




In [134]:
import os

import cv2
import numpy as np
# !pip install -U scikit-image
import pandas as pd
import torch
from PIL import Image
# !pip install -U scikit-image
from joblib import dump, load
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tqdm.notebook import tqdm
from transformers import CLIPProcessor, CLIPModel

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# !pip install keras==2.3.1

model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor_clip = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# from configs import *


def roi_highlighter(img, radious):
    if len(img.shape) == 2:  # Image is already grayscale
        gray = img
    else:
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to smooth the image and reduce noise
    gray_blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    # Detect circles in the image
    # Adjust the parameters according to your needs, especially the minRadius and maxRadius
    circles = cv2.HoughCircles(gray_blurred,
                               cv2.HOUGH_GRADIENT,
                               dp=1,
                               minDist=10,
                               param1=50,
                               param2=30,
                               minRadius=2,
                               maxRadius=24 * 2)
    # Ensure at least some circles were found
    if circles is not None:
        # Convert the circle parameters a, b, and r to integers
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the outer circle in blue color
            cv2.circle(img, (i[0], i[1]), i[2], (255, 0, 0), 2)
            # Optionally, draw the center of the circle
            cv2.circle(img, (i[0], i[1]), 2, (255, 0, 0), 3)
    return img


def extract_lbp_features(image, s=256):
    # Convert to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply LBP
    lbp = local_binary_pattern(image_gray, P=8, R=1, method="uniform")
    lbp = lbp.flatten()
    # Resize feature vector to a fixed size
    fixed_size = s  # Example size, adjust as needed
    if lbp.size < fixed_size:
        lbp = np.concatenate([lbp, np.zeros(fixed_size - lbp.size)])
    else:
        lbp = lbp[:fixed_size]
    return lbp


def extract_orb_features(image, s):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    # Standardizing to a fixed size (let's choose 256 for ORB)
    if descriptors is None:
        return np.zeros((1, s))  # Adjust size if needed
    descriptors = descriptors.flatten()[:s]  # Taking the first 256 features
    if descriptors.size < 256:
        descriptors = np.concatenate([descriptors, np.zeros(s - descriptors.size)])
    return descriptors


def extract_sift_features(img, size=128):
    # Assume img is already a grayscale image or convert it to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_img, None)
    return descriptors if descriptors is not None else np.zeros((1, size))


def extract_clip_features(img):
    # Convert cv2 BGR image to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Convert to PIL Image
    image_pil = Image.fromarray(img_rgb)
    processed = processor_clip(images=image_pil, return_tensors="pt")
    with torch.no_grad():
        features = model_clip.get_image_features(processed["pixel_values"]).squeeze(0)
    return features.cpu().numpy()


def circularity_descriptor(img, max_features=10):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 127, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Calculate circularity for each contour and sort them
    circularities = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        circularities.append(circularity)

    # Sort circularities and pad the descriptor if necessary
    circularities = sorted(circularities, reverse=True)[:max_features]
    descriptor = np.pad(circularities, (0, max_features - len(circularities)), 'constant')
    return descriptor


def extract_glcm_features(img):
    # Convert to grayscale if not already
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
    # Calculate GLCM properties
    glcm = graycomatrix(gray_img, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    features = [graycoprops(glcm, prop)[0, 0] for prop in
                ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']]
    return np.array(features)


def extract_resnet50_features(img):
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    base_model.trainable = False  # Freeze the model for feature extraction

    # Resize and preprocess the image for ResNet50
    img_resized = cv2.resize(img, (224, 224))
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB) if len(img.shape) == 3 else cv2.cvtColor(img_resized,
                                                                                                    cv2.COLOR_GRAY2RGB)
    img_array = np.expand_dims(img_rgb, axis=0)
    preprocessed_img = preprocess_input(img_array)
    features = base_model.predict(preprocessed_img)
    return features.flatten()


def combine_features(image_path, name, dt):
    sift_features = extract_sift_features(image_path, 128).flatten()[:128]  # 88
    clip_features = extract_clip_features(image_path).flatten()[:512]  # /91
    glcm_features = extract_glcm_features(image_path).flatten()  # 85
    combined_features = np.hstack(
        (sift_features, clip_features, glcm_features))  # , lbp_features, orb_features, resnet_features))
    return combined_features


def ml_features_extraction():
    train_df = pd.read_csv(train_labels_path)
    val_df = pd.read_csv(val_labels_path)

    X_train = np.array(
        [combine_features(roi_highlighter(cv2.imread(f'{train_dir}/{row.id}'), 24), row.id, "train") for _, row in
         tqdm(train_df.iterrows(), total=train_df.shape[0])])
    y_train = train_df['label'].values
    X_val = np.array(
        [combine_features(roi_highlighter(cv2.imread(f'{val_dir}/{row.id}'), 24), row.id, "val") for _, row in
         tqdm(val_df.iterrows(), total=val_df.shape[0])])
    y_val = val_df['label'].values

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    dump(scaler, os.path.join(models_path, 'scaler_model.joblib'))
    # Load the scaler model from the file
    return X_train, y_train, X_val, y_val


def test_features_extraction(scaler=None, test_dir=test_dataset_path):
    files = os.listdir(test_dir)

    files = [os.path.join(test_dir,i) for i in files]
    X_test = np.array(
        [combine_features(roi_highlighter(cv2.imread(f'{row}'), 24), row, "test") for  row in
         tqdm(files, total=len(files))])
    if scaler is None:
        scaler = load(os.path.join(models_path, 'scaler_model.joblib'))
    X_test = scaler.transform(X_test)
    return X_test


In [1]:
import numpy as np
import itertools
from sklearn.metrics import f1_score
from sklearn.ensemble import (ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier, AdaBoostClassifier)
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import NuSVC, SVC
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV, Perceptron, SGDClassifier, PassiveAggressiveClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
!pip install lightgbm
from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# 'ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'QuadraticDiscriminantAnalysis',
# 'XGBClassifier', 'DecisionTreeClassifier', 'SGDClassifier', 'dnn'

def ml_model_training_and_prediction(X_train, y_train, X_val):
    # Initialize models
    models = [
        ("ExtraTreesClassifier", ExtraTreesClassifier(**{'max_depth': 25, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 250})),
        ("LabelPropagation", LabelPropagation(**{'gamma': 100, 'kernel': 'rbf', 'n_neighbors': 17})),
        ("LabelSpreading", LabelSpreading()),
        ("LGBMClassifier", LGBMClassifier(**{'boosting_type': 'gbdt', 'learning_rate': 0.35, 'max_depth': 10, 'n_estimators': 300, 'num_leaves': 93})),
        ("AdaBoostClassifier", AdaBoostClassifier()),
        ("QuadraticDiscriminantAnalysis", QuadraticDiscriminantAnalysis(**{'reg_param': 0.0})),
        # ("XGBClassifier", XGBClassifier(**{'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 300})),
        ("DecisionTreeClassifier", DecisionTreeClassifier(**{'criterion': 'entropy', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 6, 'splitter': 'best'})),
        ("SGDClassifier", SGDClassifier()),

    ]

    # Train models and make predictions
    predictions = {}
    for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        predictions[name] = y_pred
        print(f"treaining :- {name}")


    return predictions, models


def ml_model_pred(models, X_test):
    predictions = {}
    for name, model in models:
        y_pred = model.predict(X_test)
        predictions[name] = y_pred
    return predictions

In [136]:
import itertools
import os

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report
from PIL import Image, ImageDraw, ImageFont


class PCOSTraning:

    def __init__(self):
        self.models = None
        self.dnn_model = None

    def put_text_on_image(self, image_path, text, output_path):
      image = Image.open(image_path)
      # Prepare the drawing context
      draw = ImageDraw.Draw(image)
      # Position for the text
      text_position = (25, 25)
      # Text color (RGBA)
      text_color = (255, 255, 255, 128)  # semi-transparent white text
      # Draw the text on the image
      draw.text(text_position, text, fill=text_color)
      # Save or display the image
      image.save(output_path, dpi=(600, 600))

      print("file saved successfully " , output_path)

    def plot_top_images(self, image_dir, y_pred, val_labels_path=None, dtype='val'):
      files = os.listdir(image_dir)
      df_1 = []
      df_0 = []
      if val_labels_path:
        df = pd.read_csv(val_labels_path)
        df['y_pred'] = y_pred
        df['label']= df['label'].apply(str)
        df['y_pred']= df['y_pred'].apply(str)
        df= df[df['label']==df['y_pred']]
        df_1 = df[df['y_pred']=="1"]['id'].to_list()[:2]
        df_0 = df[df['y_pred']=="0"]['id'].to_list()[:3]
      else:
        df = pd.DataFrame({'id':files, 'label':y_pred})
        df['label']= df['label'].apply(str)
        df_1 = df[df['label']=="1"]['id'].to_list()[:2]
        df_0 = df[df['label']=="0"]['id'].to_list()[:3]
      z= {'class_1':df_1, 'class_0':df_0}
      for c in z:
        for img in z[c]:
          self.put_text_on_image(
              os.path.join(image_dir, img),
              c, os.path.join(plot_dir, f'{dtype}_sample'+img)
          )




    def dnn_and_ml_ensemble_eval(self, predictions, y_val, models):

        ensemble_pred = []
        for j in range(len(models), len(models) + 1):  # Start from 1 to include at least one model besides GaussianNB
            comb_scores = {}
            for combo in itertools.combinations([model[0] for model in models],
                                                j - 1):  # Adjust to j - 1 to leave space for GaussianNB
                combo = combo + ("dnn",)  # Always include GaussianNB in the combination
                # Ensemble prediction by majority vote, now including GaussianNB
                ensemble_pred = np.array([np.bincount(
                    [predictions[model_name][i] for model_name in combo]
                ).argmax() for i in range(len(y_val))])
                f1 = f1_score(y_val, ensemble_pred, average='macro')
                comb_scores[combo] = f1

            # Sort combinations by F1 score
            sorted_comb_scores = sorted(comb_scores.items(), key=lambda x: x[1], reverse=True)

            # Print the best combination for this round
            best_combo, best_score = sorted_comb_scores[0]
            print(f"F1 score: {best_score} with Best combination: {best_combo}")

        print(classification_report(y_val, ensemble_pred))
        return ensemble_pred

    def dnn_and_ml_ensemble_pred(self, predictions, models):
        ensemble_pred=[]
        for j in range(len(models), len(models) + 1):  # Start from 1 to include at least one model besides GaussianNB
            comb_scores = {}
            for combo in itertools.combinations([model[0] for model in models],
                                                j - 1):  # Adjust to j - 1 to leave space for GaussianNB
                combo = combo + ("dnn",)  # Always include GaussianNB in the combination
                # Ensemble prediction by majority vote, now including GaussianNB
                ensemble_pred = np.array([np.bincount(
                    [predictions[model_name][i] for model_name in combo]
                ).argmax() for i in range(len(predictions['dnn']))])
        return ensemble_pred

    def main(self):
        prepare_balanced_dataset()
        print("start Deep Neural Network Model Training....")
        dnn_model = dnn_model_train_and_eval()
        dnn_model=None
        dnn_y_pred = dnn_predict(val_dir, val_labels_path, dnn_model, dtype='val')
        print("start feature extraction for ML models....")

        X_train, y_train, X_val, y_val = ml_features_extraction()
        print("start ML models training....")

        predictions, models = ml_model_training_and_prediction(X_train, y_train, X_val)
        predictions['dnn'] = dnn_y_pred
        ensemble_pred = self.dnn_and_ml_ensemble_eval(predictions, y_val, models)
        self.models = models
        self.dnn_model = dnn_model
        self.plot_top_images(val_dir, ensemble_pred, val_labels_path, dtype='val')

    def test_label_prediction(self, test_dir=test_dataset_path):

        dnn_test_prediction = dnn_predict(test_dir, labels_file_path=None, loaded_model=self.dnn_model, dtype = 'test')
        X_test = test_features_extraction(scaler=None, test_dir=test_dataset_path)


        ml_test_predictions = ml_model_pred(self.models, X_test)
        ml_test_predictions['dnn'] = dnn_test_prediction
        predictions = self.dnn_and_ml_ensemble_pred(ml_test_predictions, self.models)
        df = pd.DataFrame({"S.No.": list(range(1,len(predictions)+1)),"Image Path": os.listdir(test_dir), "Predicted Class Label": predictions})
        df.to_excel(result_submission_file, index=False)
        self.plot_top_images(test_dir, predictions, None, dtype='test')



obj = PCOSTraning()
obj.main()
obj.test_label_prediction(test_dataset_path)

/content/drive/MyDrive/data/input/PCOSGen-test/images/
start Deep Neural Network Model Training....


Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
LINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use BILINEAR or Resampling.BILINEAR instead.


start feature extraction for ML models....


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
`max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.


treaining :- ExtraTreesClassifier


...

treaining :- QuadraticDiscriminantAnalysis
treaining :- DecisionTreeClassifier


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


treaining :- SGDClassifier
F1 score: 0.9268952350648317 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'LGBMClassifier', 'AdaBoostClassifier', 'QuadraticDiscriminantAnalysis', 'DecisionTreeClassifier', 'dnn')
              precision    recall  f1-score   support

           0       0.88      0.97      0.93       460
           1       0.97      0.87      0.92       459

    accuracy                           0.92       919
   macro avg       0.93      0.92      0.92       919
weighted avg       0.93      0.92      0.92       919

file saved successfully  /content/drive/MyDrive/data/output/plots/val_sampleimage0758.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/val_sampleimage1645.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/val_sampleimage0718.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/val_sampleimage0304.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
LINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use BILINEAR or Resampling.BILINEAR instead.
invalid value encountered in divide
invalid value encountered in divide


file saved successfully  /content/drive/MyDrive/data/output/plots/test_sampleimage10906.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/test_sampleimage10536.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/test_sampleimage10234.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/test_sampleimage10603.jpg
file saved successfully  /content/drive/MyDrive/data/output/plots/test_sampleimage10466.jpg


In [137]:
# F1 score: 0.7285210596531351 with Best combination: ('dnn',)
# F1 score: 0.8466243868458481 with Best combination: ('LabelPropagation', 'dnn')
# F1 score: 0.9301087535173778 with Best combination: ('LabelPropagation', 'LabelSpreading', 'dnn')
# F1 score: 0.9301087535173778 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'dnn')
# F1 score: 0.9302579280381722 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'AdaBoostClassifier', 'dnn')
# F1 score: 0.9324258294670728 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'LGBMClassifier', 'AdaBoostClassifier', 'dnn')
# F1 score: 0.9323873399215865 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'QuadraticDiscriminantAnalysis', 'AdaBoostClassifier', 'PassiveAggressiveClassifier', 'dnn')
# add Codeadd Markdown

# F1 score: 0.7445257035326318 with Best combination: ('dnn',)
# F1 score: 0.8477845870756597 with Best combination: ('LabelPropagation', 'dnn')
# F1 score: 0.9301087535173778 with Best combination: ('LabelPropagation', 'LabelSpreading', 'dnn')
# F1 score: 0.9301087535173778 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'dnn')
# F1 score: 0.9301087535173778 with Best combination: ('LabelPropagation', 'LabelSpreading', 'QuadraticDiscriminantAnalysis', 'DecisionTreeClassifier', 'dnn')
# F1 score: 0.930220197418375 with Best combination: ('ExtraTreesClassifier', 'LabelPropagation', 'LabelSpreading', 'LGBMClassifier', 'AdaBoostClassifier', 'dnn')